# Extract Dataset Archives

In [1]:
import tarfile

# Extract images.tar.gz
with tarfile.open("OxfordPets/images.tar.gz", "r:gz") as tar:
    tar.extractall(path="OxfordPets/")

# Extract annotations.tar.gz
with tarfile.open("OxfordPets/annotations.tar.gz", "r:gz") as tar:
    tar.extractall(path="OxfordPets/")


C:\Users\joelv\AppData\Local\Temp\ipykernel_31140\3555206837.py:5: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="OxfordPets/")
C:\Users\joelv\AppData\Local\Temp\ipykernel_31140\3555206837.py:9: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path="OxfordPets/")


# Convert Annotations to YOLO Format

In [3]:
import os
import xml.etree.ElementTree as ET

# Paths
image_folder = "OxfordPets/images"
annotation_folder = "OxfordPets/annotations"
yolo_labels_folder = "OxfordPets/labels"

# Ensure the labels folder exists
os.makedirs(yolo_labels_folder, exist_ok=True)

# Define class mapping
class_mapping = {"cat": 0, "dog": 1}  # Assign class IDs

# Process each annotation file
for xml_file in os.listdir(annotation_folder):
    if not xml_file.endswith(".xml"):
        continue  # Skip non-XML files

    tree = ET.parse(os.path.join(annotation_folder, xml_file))
    root = tree.getroot()

    image_name = root.find("filename").text
    image_path = os.path.join(image_folder, image_name)
    label_path = os.path.join(yolo_labels_folder, image_name.replace(".jpg", ".txt"))

    # Read image dimensions
    img_w = int(root.find("size/width").text)
    img_h = int(root.find("size/height").text)

    with open(label_path, "w") as f:
        for obj in root.findall("object"):
            class_name = obj.find("name").text.lower()
            if class_name not in class_mapping:
                continue  # Ignore non-cat labels

            class_id = class_mapping[class_name]
            bbox = obj.find("bndbox")

            xmin = int(bbox.find("xmin").text)
            ymin = int(bbox.find("ymin").text)
            xmax = int(bbox.find("xmax").text)
            ymax = int(bbox.find("ymax").text)

            # Convert to YOLO format (normalize)
            x_center = (xmin + xmax) / (2 * img_w)
            y_center = (ymin + ymax) / (2 * img_h)
            width = (xmax - xmin) / img_w
            height = (ymax - ymin) / img_h

            f.write(f"{class_id} {x_center} {y_center} {width} {height}\n")


# Check for Annotations

In [5]:
import os

annotation_folder = "OxfordPets/annotations"
annotation_files = os.listdir(annotation_folder)

if annotation_files:
    print(f"Found {len(annotation_files)} annotation files.")
    print("First few annotation files:", annotation_files[:5])
else:
    print("No annotation files found! Check if the dataset extracted correctly.")


Found 7 annotation files.
First few annotation files: ['._trimaps', 'list.txt', 'README', 'test.txt', 'trainval.txt']


# Preview Contents and Trimaps

In [7]:
with open(os.path.join(annotation_folder, "list.txt"), "r") as f:
    print(f.read()[:1000])  # Print the first 1000 characters
    
trimaps_folder = os.path.join(annotation_folder, "trimaps")
if os.path.exists(trimaps_folder):
    print("Trimaps found:", os.listdir(trimaps_folder)[:5])
else:
    print("Trimaps folder does not exist!")


#Image CLASS-ID SPECIES BREED ID
#ID: 1:37 Class ids
#SPECIES: 1:Cat 2:Dog
#BREED ID: 1-25:Cat 1:12:Dog
#All images with 1st letter as captial are cat images
#images with small first letter are dog images
Abyssinian_100 1 1 1
Abyssinian_101 1 1 1
Abyssinian_102 1 1 1
Abyssinian_103 1 1 1
Abyssinian_104 1 1 1
Abyssinian_105 1 1 1
Abyssinian_106 1 1 1
Abyssinian_107 1 1 1
Abyssinian_108 1 1 1
Abyssinian_109 1 1 1
Abyssinian_10 1 1 1
Abyssinian_110 1 1 1
Abyssinian_111 1 1 1
Abyssinian_112 1 1 1
Abyssinian_113 1 1 1
Abyssinian_114 1 1 1
Abyssinian_115 1 1 1
Abyssinian_116 1 1 1
Abyssinian_117 1 1 1
Abyssinian_118 1 1 1
Abyssinian_119 1 1 1
Abyssinian_11 1 1 1
Abyssinian_120 1 1 1
Abyssinian_121 1 1 1
Abyssinian_122 1 1 1
Abyssinian_123 1 1 1
Abyssinian_124 1 1 1
Abyssinian_125 1 1 1
Abyssinian_126 1 1 1
Abyssinian_127 1 1 1
Abyssinian_128 1 1 1
Abyssinian_129 1 1 1
Abyssinian_12 1 1 1
Abyssinian_130 1 1 1
Abyssinian_131 1 1 1
Abyssinian_132 1 1 1
Abyssinian_133 1 1 1
Abyssinian_134 1 1 1


In [9]:
import os

# Path to the XML files folder
xml_folder = "OxfordPets/annotations/xmls"
xml_files = os.listdir(xml_folder)

# Print the first few XML files
print(f"Found {len(xml_files)} XML files.")
print("First few XML files:", xml_files[:5])


Found 3686 XML files.
First few XML files: ['Abyssinian_1.xml', 'Abyssinian_10.xml', 'Abyssinian_100.xml', 'Abyssinian_101.xml', 'Abyssinian_102.xml']


# Inspect a Sample Annotation File

In [11]:
import xml.etree.ElementTree as ET

# Path to one of the XML files
sample_xml_file = os.path.join(xml_folder, 'Abyssinian_1.xml')

# Parse the XML file
tree = ET.parse(sample_xml_file)
root = tree.getroot()

# Print the structure of the XML file to verify the content
ET.dump(root)  # This will print the entire XML content


<annotation><folder>OXIIIT</folder><filename>Abyssinian_1.jpg</filename><source><database>OXFORD-IIIT Pet Dataset</database><annotation>OXIIIT</annotation><image>flickr</image></source><size><width>600</width><height>400</height><depth>3</depth></size><segmented>0</segmented><object><name>cat</name><pose>Frontal</pose><truncated>0</truncated><occluded>0</occluded><bndbox><xmin>333</xmin><ymin>72</ymin><xmax>425</xmax><ymax>158</ymax></bndbox><difficult>0</difficult></object></annotation>


In [13]:
import os
import xml.etree.ElementTree as ET

# Paths
image_folder = "OxfordPets/images"
annotation_folder = "OxfordPets/annotations/xmls"
yolo_labels_folder = "OxfordPets/labels"

# Ensure the labels folder exists
os.makedirs(yolo_labels_folder, exist_ok=True)

# Class mapping: Assuming 'cat' = 0
class_mapping = {"cat": 0}

# Process each XML file in the xmls folder
for xml_file in os.listdir(annotation_folder):
    if not xml_file.endswith(".xml"):
        continue  # Skip non-XML files

    xml_path = os.path.join(annotation_folder, xml_file)

    try:
        # Parse XML file
        tree = ET.parse(xml_path)
        root = tree.getroot()

        # Get image filename (e.g., Abyssinian_1.jpg)
        image_name = root.find("filename").text
        label_filename = image_name.replace(".jpg", ".txt")
        label_path = os.path.join(yolo_labels_folder, label_filename)

        # Get image dimensions
        img_w = int(root.find("size/width").text)
        img_h = int(root.find("size/height").text)

        # Open the YOLO label file
        with open(label_path, "w") as label_file:
            for obj in root.findall("object"):
                class_name = obj.find("name").text.lower()
                if class_name not in class_mapping:
                    continue  # Ignore non-cat objects

                class_id = class_mapping[class_name]
                bbox = obj.find("bndbox")

                # Extract bounding box coordinates
                xmin = int(bbox.find("xmin").text)
                ymin = int(bbox.find("ymin").text)
                xmax = int(bbox.find("xmax").text)
                ymax = int(bbox.find("ymax").text)

                # Normalize bounding box
                x_center = (xmin + xmax) / (2 * img_w)
                y_center = (ymin + ymax) / (2 * img_h)
                width = (xmax - xmin) / img_w
                height = (ymax - ymin) / img_h

                # Write to YOLO label file
                label_file.write(f"{class_id} {x_center} {y_center} {width} {height}\n")

    except ET.ParseError as e:
        print(f"Error parsing {xml_file}: {e}")
    except Exception as e:
        print(f"Unexpected error in {xml_file}: {e}")

print(" YOLO conversion complete!")


 YOLO annotation conversion complete!


# Split Dataset into training and validation

In [14]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Paths to images and labels
image_folder = "OxfordPets/images"
label_folder = "OxfordPets/labels"

# Create train and val directories
train_image_folder = "OxfordPets/images/train"
val_image_folder = "OxfordPets/images/val"
train_label_folder = "OxfordPets/labels/train"
val_label_folder = "OxfordPets/labels/val"

os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(val_image_folder, exist_ok=True)
os.makedirs(train_label_folder, exist_ok=True)
os.makedirs(val_label_folder, exist_ok=True)

# Get list of image files (ensure label exists)
image_files = [f.replace(".jpg", "") for f in os.listdir(image_folder) 
               if f.endswith(".jpg") and os.path.exists(os.path.join(label_folder, f.replace(".jpg", ".txt")))]

# Split data into training (80%) and validation (20%)
train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

# Copy images and labels to train/val folders
for file in train_files:
    shutil.copy(os.path.join(image_folder, file + ".jpg"), os.path.join(train_image_folder, file + ".jpg"))
    shutil.copy(os.path.join(label_folder, file + ".txt"), os.path.join(train_label_folder, file + ".txt"))

for file in val_files:
    shutil.copy(os.path.join(image_folder, file + ".jpg"), os.path.join(val_image_folder, file + ".jpg"))
    shutil.copy(os.path.join(label_folder, file + ".txt"), os.path.join(val_label_folder, file + ".txt"))

print(" Dataset successfully split into train and val folders.")


 Dataset successfully split into train and val folders.


# Install Ultralytics

In [1]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


# Model Training 

In [15]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO("yolov8n.yaml")

# Train the model
model.train(data="OxfordPets/data.yaml", epochs=50, imgsz=640)


New https://pypi.org/project/ultralytics/8.3.126 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.107  Python-3.12.7 torch-2.6.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=OxfordPets/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train18, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_tx

train: Scanning C:\Users\joelv\Desktop\Project\OxfordPets\labels\train... 3348 images, 2291 backgrounds, 0 corrupt: 100

train: WARNING  C:\Users\joelv\Desktop\Project\OxfordPets\images\train\Egyptian_Mau_138.jpg: corrupt JPEG restored and saved


train: New cache created: C:\Users\joelv\Desktop\Project\OxfordPets\labels\train.cache


val: Scanning C:\Users\joelv\Desktop\Project\OxfordPets\labels\val.cache... 1136 images, 809 backgrounds, 0 corrupt: 10


...

                   all       1136        328      0.978      0.971      0.986      0.899



50 epochs completed in 23.823 hours.
Optimizer stripped from runs\detect\train18\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train18\weights\best.pt, 6.2MB

Validating runs\detect\train18\weights\best.pt...
Ultralytics 8.3.107  Python-3.12.7 torch-2.6.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
YOLOv8n summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [01:59


                   all       1136        328      0.975      0.979      0.986      0.899
Speed: 2.1ms preprocess, 92.2ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train18


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002510C01B8C0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

# Evaluate Model Performance

In [25]:
metrics = model.val()

precision_mean = metrics.box.mp       # mean precision across classes
recall_mean    = metrics.box.mr       # mean recall across classes
map50          = metrics.box.map50    # mAP @ IoU=0.50
map50_95       = metrics.box.map      # mAP @ IoU=0.50:0.95
fitness_score  = metrics.fitness      # single fitness value

print(f"Precision: {precision_mean:.4f}")
print(f"Recall:    {recall_mean:.4f}")
print(f"mAP@0.5:   {map50:.4f}")
print(f"mAP@.5:.95:{map50_95:.4f}")
print(f"Fitness:  {fitness_score:.4f}")


Ultralytics 8.3.107  Python-3.12.7 torch-2.6.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)


val: Scanning C:\Users\joelv\Desktop\Project\OxfordPets\labels\val.cache... 1136 images, 809 backgrounds, 0 corrupt: 10
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 71/71 [01:39


                   all       1136        328      0.972      0.976      0.986      0.896
Speed: 1.2ms preprocess, 75.3ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs\detect\train185
Precision: 0.9725
Recall:    0.9756
mAP@0.5:   0.9860
mAP@.5:.95:0.8965
Fitness:  0.9054


# Run Inference on a Video and Save Output

In [ ]:
import cv2
from ultralytics import YOLO

# Load trained model
model = YOLO("runs/detect/train/weights/best.pt")

# Open video file
cap = cv2.VideoCapture("test_videos/sample.mp4")

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define video writer
out = cv2.VideoWriter("output.mp4", cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame)

    for r in results:
        for box in r.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    out.write(frame)
    cv2.imshow("Frame", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
out.release()
cv2.destroyAllWindows()



0: 640x384 1 cat, 98.7ms
Speed: 0.0ms preprocess, 98.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cat, 87.8ms
Speed: 5.0ms preprocess, 87.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cat, 84.5ms
Speed: 4.0ms preprocess, 84.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cat, 90.6ms
Speed: 3.0ms preprocess, 90.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cat, 81.4ms
Speed: 15.2ms preprocess, 81.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cat, 114.5ms
Speed: 18.9ms preprocess, 114.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cat, 94.5ms
Speed: 15.1ms preprocess, 94.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 1 cat, 83.4ms
Speed: 1.3ms preprocess, 83.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384